In [4]:
import pybikes
import delta
import requests

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from delta import *

In [5]:
spark = SparkSession.builder.getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx12884m
Picked up JAVA_TOOL_OPTIONS:  -Xmx12884m
23/06/28 19:05:03 WARN Utils: Your hostname, thpereiradu-datapipelin-7eb5vckxjza resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface eth0)
23/06/28 19:05:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/28 19:05:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# NETWORK TABLE

response = requests.get("https://api.citybik.es/v2/networks")
data = response.json()
rdd = spark.sparkContext.parallelize(data["networks"])
df_networks = rdd.toDF()
df_networks.show()

+--------------------+--------------------+--------------------+--------------------+----------+
|             company|                href|                  id|            location|      name|
+--------------------+--------------------+--------------------+--------------------+----------+
|    [ЗАО «СитиБайк»]|/v2/networks/velo...|     velobike-moscow|{country -> RU, c...|  Velobike|
|[Urban Infrastruc...|/v2/networks/baer...|     baerum-bysykkel|{country -> NO, c...|  Bysykkel|
| [Comunicare S.r.l.]|/v2/networks/bici...|    bicincitta-siena|{country -> IT, c...|Bicincittà|
|[Cyclopolis Systems]|/v2/networks/cycl...| cyclopolis-maroussi|{country -> GR, c...|Cyclopolis|
|[Groundwork, Slou...|/v2/networks/cycl...|   cycle-hire-slough|{country -> GB, c...|Cycle Hire|
|[Cyclopolis Systems]|/v2/networks/cycl...|  cyclopolis-nafplio|{country -> GR, c...|Cyclopolis|
| [Comunicare S.r.l.]|/v2/networks/bici...|bicincitta-parco-...|{country -> IT, c...|Bicincittà|
|[Cyclopolis Systems]|/v2/netw

In [7]:
# EXCEPTION IDs LIST

exception_ids = []

unique_ids = df_networks.select('id').distinct().rdd.flatMap(lambda x: x).collect()

for id in unique_ids:
    try:
        id_instance = pybikes.get(id)
        id_instance.update()
    except Exception as e:
        print(f"Erro ao atualizar dados para a estação {id}: {e}")
        exception_ids.append(id)

        exception_ids

Erro ao atualizar dados para a estação wowcycle-reykjavik: HTTPSConnectionPool(host='rey.publicbikesystem.net', port=443): Max retries exceeded with url: /ube/stations (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f85b5323c90>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Erro ao atualizar dados para a estação punpunbikeshare: HTTPConnectionPool(host='pcc.punpunbikeshare.com', port=80): Max retries exceeded with url: /index.php?r=api2/listStations&token=1111&key=3e5cc0883cf7134b6b3ea7157d2cb4ef (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f85b53a2450>, 'Connection to pcc.punpunbikeshare.com timed out. (connect timeout=300)'))
Erro ao atualizar dados para a estação bbbike: HTTPSConnectionPool(host='bbbike.eu', port=443): Max retries exceeded with url: /mapa-stacji.html (Caused by SSLError(CertificateError("hostname 'bbbike.eu' doesn't match either of '*.home.pl', 'home.pl'")))
Erro ao at

/home/gitpod/.pyenv/versions/3.11.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rivibike.tembici.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Erro ao atualizar dados para a estação rivibike: Expecting value: line 1 column 1 (char 0)
Erro ao atualizar dados para a estação velostanlib: System Cyclocity needs a key to work
Erro ao atualizar dados para a estação velocite-besancon: System Cyclocity needs a key to work
Erro ao atualizar dados para a estação velo2: System Cyclocity needs a key to work
Erro ao atualizar dados para a estação cristolib: System Cyclocity needs a key to work
Erro ao atualizar dados para a estação baksi-istanbul: HTTPSConnectionPool(host='www.isbike.com.tr', port=443): Max retries exceeded with url: /abone/harita.asp (Caused by SSLError(CertificateError("hostname 'www.isbike.com.tr' doesn't match either of '*.isbike.istanbul', 'isbike.istanbul'")))
Erro ao atualizar dados para a estação baksi-kobis: HTTPConnectionPool(host='abone.kobis.com.tr', port=80): Max retries exceeded with url: /harita.asp (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f85b4f9e7d0>, 'Connection to a

In [8]:
# ONLINE IDs FILTER

df_networks_on = df_networks.filter(~col("id").isin(exception_ids))
df_networks_on.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|             company|                href|                  id|            location|                name|
+--------------------+--------------------+--------------------+--------------------+--------------------+
| [Comunicare S.r.l.]|/v2/networks/bici...|    bicincitta-siena|{country -> IT, c...|          Bicincittà|
|[Cyclopolis Systems]|/v2/networks/cycl...| cyclopolis-maroussi|{country -> GR, c...|          Cyclopolis|
|[Cyclopolis Systems]|/v2/networks/cycl...|  cyclopolis-nafplio|{country -> GR, c...|          Cyclopolis|
| [Comunicare S.r.l.]|/v2/networks/bici...|bicincitta-parco-...|{country -> IT, c...|          Bicincittà|
|[Cyclopolis Systems]|/v2/networks/cycl...|cyclopolis-aigialeia|{country -> GR, c...|          Cyclopolis|
|[Cyclopolis Systems]|/v2/networks/cycl...| cyclopolis-marathon|{country -> GR, c...|          Cyclopolis|
| [Comunicare S.r.l.]|/v2/networks/bi

In [9]:
# STATIONS TABLE
import json

ids_list = [row.id for row in df_networks_on.collect()]

response_data = []

for id in ids_list:
    url = f"https://api.citybik.es/v2/networks/{id}"
    response = requests.get(url)
    if response.status_code == 200:
        response_data.append(response.json())

schema = "response STRING"
data_rdd = spark.sparkContext.parallelize([(json.dumps(data),) for data in response_data])
df_stations = spark.createDataFrame(data=data_rdd, schema=schema)

df_stations.show()

+--------------------+
|            response|
+--------------------+
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
|{"network": {"com...|
+--------------------+
only showing top 20 rows



In [10]:
# STATIONS TABLE STRUCTURE

schema = StructType([
    StructField("network", StructType([
        StructField("company", ArrayType(StringType())),
        StructField("href", StringType()),
        StructField("id", StringType()),
        StructField("location", StructType([
            StructField("city", StringType()),
            StructField("country", StringType()),
            StructField("latitude", DoubleType()),
            StructField("longitude", DoubleType())
        ])),
        StructField("name", StringType()),
        StructField("source", StringType()),
        StructField("stations", ArrayType(StructType([
            StructField("empty_slots", IntegerType()),
            StructField("extra", StructType([
                StructField("number", IntegerType()),
                StructField("reviews", IntegerType()),
                StructField("score", DoubleType()),
                StructField("status", StringType()),
                StructField("uid", StringType())
            ])),
            StructField("free_bikes", IntegerType()),
            StructField("id", StringType()),
            StructField("latitude", DoubleType()),
            StructField("longitude", DoubleType()),
            StructField("name", StringType()),
            StructField("timestamp", StringType())
        ])))
    ]))
])


df_stations = df_stations.selectExpr("CAST(response AS STRING)").select(
    from_json("response", schema).alias("data")
).select(
    col("data.network.company").alias("company"),
    col("data.network.href").alias("href"),
    col("data.network.id").alias("id"),
    col("data.network.location.city").alias("city"),
    col("data.network.location.country").alias("country"),
    col("data.network.location.latitude").alias("latitude"),
    col("data.network.location.longitude").alias("longitude"),
    col("data.network.name").alias("name"),
    col("data.network.source").alias("source"),
    explode(col("data.network.stations")).alias("stations")
).select(
    col("company"),
    col("href"),
    col("id"),
    col("city"),
    col("country"),
    col("latitude"),
    col("longitude"),
    col("name"),
    col("source"),
    col("stations.empty_slots").alias("empty_slots"),
    col("stations.extra.number").alias("extra_number"),
    col("stations.extra.reviews").alias("extra_reviews"),
    col("stations.extra.score").alias("extra_score"),
    col("stations.extra.status").alias("extra_status"),
    col("stations.extra.uid").alias("extra_uid"),
    col("stations.free_bikes").alias("free_bikes"),
    col("stations.id").alias("station_id"),
    col("stations.latitude").alias("station_latitude"),
    col("stations.longitude").alias("station_longitude"),
    col("stations.name").alias("station_name"),
    col("stations.timestamp").alias("timestamp")
)


df_stations.show()


+--------------------+--------------------+-------------------+--------+-------+-------------+-------------+----------+--------------------+-----------+------------+-------------+-----------+------------+---------+----------+--------------------+------------------+------------------+--------------------+--------------------+
|             company|                href|                 id|    city|country|     latitude|    longitude|      name|              source|empty_slots|extra_number|extra_reviews|extra_score|extra_status|extra_uid|free_bikes|          station_id|  station_latitude| station_longitude|        station_name|           timestamp|
+--------------------+--------------------+-------------------+--------+-------+-------------+-------------+----------+--------------------+-----------+------------+-------------+-----------+------------+---------+----------+--------------------+------------------+------------------+--------------------+--------------------+
| [Comunicare S.r.l